In [23]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation, BatchNormalization
from tensorflow.keras.utils import plot_model,to_categorical
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sn

In [24]:
#Creation of DataFrame
df_train = pd.read_csv("MURA-v1.1/train_image_paths.csv", header=None)
df_val = pd.read_csv("MURA-v1.1/valid_image_paths.csv", header=None)
df = pd.concat([df_train, df_val], axis=0, ignore_index = True)
#Split image path
imageSplit= df[0].str.split('/', n=5, expand=True)
df['ImageSet'] = imageSplit[1]
df['ImageType'] = imageSplit[2]
df['Diagnosis'] = imageSplit[4].str.split('_', n=1, expand=True)[1]
df = df.rename(columns={0: 'ImagePath'})
#Change positives to 1s and Negatives to 0
df.loc[df['Diagnosis'] == 'positive', 'Diagnosis'] = 1
df.loc[df['Diagnosis'] == 'negative', 'Diagnosis'] = 0

#Deletion of extra variables to free space
del df_train, df_val, imageSplit
df

,ImagePath,ImageSet,ImageType,Diagnosis
0,MURA-v1.1/train/XR_SHOULDER/patient00001/study...,train,XR_SHOULDER,1
1,MURA-v1.1/train/XR_SHOULDER/patient00001/study...,train,XR_SHOULDER,1
2,MURA-v1.1/train/XR_SHOULDER/patient00001/study...,train,XR_SHOULDER,1
3,MURA-v1.1/train/XR_SHOULDER/patient00002/study...,train,XR_SHOULDER,1
4,MURA-v1.1/train/XR_SHOULDER/patient00002/study...,train,XR_SHOULDER,1
...,...,...,...,...
40000,MURA-v1.1/valid/XR_FINGER/patient11967/study1_...,valid,XR_FINGER,0
40001,MURA-v1.1/valid/XR_FINGER/patient11967/study1_...,valid,XR_FINGER,0
40002,MURA-v1.1/valid/XR_FINGER/patient11738/study1_...,valid,XR_FINGER,0
40003,MURA-v1.1/valid/XR_FINGER/patient11738/study1_...,valid,XR_FINGER,0


In [25]:
df_train = df.loc[(df['ImageType'] == 'XR_SHOULDER') & (df['ImageSet'] == 'train')]

train_imagePath = df_train['ImagePath'].to_list()
train_labels = df_train['Diagnosis'].to_list()

df_valid = df.loc[(df['ImageType'] == 'XR_SHOULDER') & (df['ImageSet'] == 'valid')]
val_imagePath = df_valid['ImagePath'].to_list()
val_labels = df_valid['Diagnosis'].to_list()

In [26]:
def convertImage(imagePaths):
    images = []
    for x in imagePaths:
        image = cv2.imread(x)
        image = cv2.resize(image, (227,227))
        if image.shape[2] == 1:
            img = np.dstack([image,image,image])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = image.astype(np.float32)/255
        images.append(image)
    print("Images Converted to cv2")
    return images
    

train_images = convertImage(train_imagePath)
val_images = convertImage(val_imagePath)

Images Converted to cv2
Images Converted to cv2


In [47]:
#Creation of Alexnet Architecture
model = keras.models.Sequential()
num_output = 2
model.add(Conv2D(filters = 96, input_shape = (227, 227, 3), 
            kernel_size = (11, 11), strides = (4, 4), 
            padding = 'valid'))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size =(2,2), strides = (2,2), padding='valid'))

model.add(BatchNormalization())

model.add(Conv2D(filters = 256, kernel_size = (11, 11), 
            strides = (1, 1), padding = 'valid'))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), 
            padding = 'valid'))

model.add(BatchNormalization())

model.add(Conv2D(filters = 384, kernel_size = (3, 3), 
            strides = (1, 1), padding = 'valid'))

model.add(Activation('relu'))

model.add(BatchNormalization())

model.add(Conv2D(filters = 384, kernel_size = (3, 3), 
            strides = (1, 1), padding = 'valid'))

model.add(Activation('relu'))

model.add(BatchNormalization())

model.add(Conv2D(filters = 256, kernel_size = (3, 3), 
            strides = (1, 1), padding = 'valid'))

model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), 
            padding = 'valid'))

model.add(BatchNormalization())

model.add(Flatten())

model.add(Dense(4096, input_shape = (224*224*3, )))
model.add(Activation('relu'))

model.add(Dropout(0.4))

model.add(BatchNormalization())

model.add(Dense(4096))
model.add(Activation('relu'))


model.add(Dropout(0.4))

model.add(BatchNormalization())
  

model.add(Dense(num_output))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [45]:
train_images = np.array(train_images)
val_images = np.array(val_images)
train_labels = to_categorical(train_labels, num_classes=2)
val_labels = to_categorical(val_labels, num_classes=2)

In [48]:
history = model.fit(train_images, train_labels, batch_size = 512, epochs = 50, validation_data = (val_images, val_labels))

Epoch 1/50
17/17 [==============================] - 109s 6s/step - loss: 3.2338 - accuracy: 0.5046 - val_loss: 2.0835 - val_accuracy: 0.4902
Epoch 2/50
17/17 [==============================] - 107s 6s/step - loss: 0.8697 - accuracy: 0.5176 - val_loss: 2.3775 - val_accuracy: 0.4938
Epoch 3/50
17/17 [==============================] - 114s 7s/step - loss: 0.7762 - accuracy: 0.5353 - val_loss: 2.9686 - val_accuracy: 0.4991
Epoch 4/50
17/17 [==============================] - 115s 7s/step - loss: 0.7585 - accuracy: 0.5336 - val_loss: 0.7496 - val_accuracy: 0.5151
Epoch 5/50
17/17 [==============================] - 116s 7s/step - loss: 0.7458 - accuracy: 0.5372 - val_loss: 0.7370 - val_accuracy: 0.4991
Epoch 6/50
17/17 [==============================] - 118s 7s/step - loss: 0.7431 - accuracy: 0.5447 - val_loss: 0.7385 - val_accuracy: 0.4654
Epoch 7/50
17/17 [==============================] - 110s 6s/step - loss: 0.7228 - accuracy: 0.5668 - val_loss: 0.6887 - val_accuracy: 0.5471
Epoch 8/50
17